In [2]:
import pandas as pd
import networkx as nx
import numpy as np 

In [3]:
reads = pd.read_csv('TestFile_frag.csv')

In [4]:
reads.head()

,Unnamed: 0,Amplicon,Query,Molecule,Start,End,count
0,0,2,0,.................000000000000000..,17,32,1
1,1,2,0,00000000000000000.................,0,17,1
2,2,3,0,00000000000.......................,0,11,1
3,3,4,0,00000000000.......................,0,11,1
4,4,5,0,......................000000000000,22,34,1


In [18]:
molecules = list(reads['Molecule'])
# print(molecules)

def translate(x): 
    CONVERT = {'0':0,'.':np.nan,'C':1,'A':1,'T':1,'G':1}
    return CONVERT[x]

np_mols = np.array([list(map(translate, x)) for x in molecules])


In [19]:
np_mols[1]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan])

In [22]:
np_mols[54]

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.])

In [28]:
np.sum([np_mols[1],np_mols[54]],axis=0)

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,   0.,   0.,   0.,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan])

In [131]:
molecules = list(reads['Molecule'])
def translate(x): 
    CONVERT = {'0':0,'.':np.nan,'C':1,'A':1,'T':1,'G':1}
    return CONVERT[x]

np_mols = np.array([list(map(translate, x)) for x in molecules])

starts = list(reads['Start'])
ends = [x for x in list(reads['End'])]
clusters= [0]*len(molecules)

# complete = []
N = len(molecules)

import copy

def get_overlap(coords_1, coords_2):
    '''
    @param: coords_1 - tuple contain (start,end) of molecule1
    @param: coords_2 - tuple contain (start,end) of molecule2
    '''
    return set(range(coords_1[0],coords_1[1])).intersection(range(coords_2[0],coords_2[1]))

max_overlap = 1
iters = 0
while max_overlap > 0 and len(molecules)>0:
    iters +=1 
    merge = (0,0)
    max_overlap = 0 
    for i in range(len(molecules)):
        for j in range(i+1, len(molecules)):
            if molecules[i] != molecules[j]:
                mol1 = molecules[i][starts[i]:ends[i]]
                mol2 = molecules[j][starts[j]:ends[j]]
                
                print(mol1)
                print(mol2)
#                 print('____')
                overlap = get_overlap((starts[i],ends[i]+1),(starts[j],ends[j]+1))
                print(starts[i],ends[i])
                print(starts[j],ends[j])
                if len(overlap)==0:
                    continue
                print(mol1)
                print(mol2)
                print(overlap)

                start = min(overlap)
                end = max(overlap)

                mol1 = molecules[i][start:end]
                mol2 = molecules[j][start:end]
                
                print(mol1)
                print(mol2)
                print('____')
                if mol1==mol2:
                    if overlaps[i,j] > max_overlap:
                        max_overlap = overlaps[i,j]
                        merge=(i,j)
    # Merge the reads with the greatest overlap
    
    #merge[0] < merge[1] always
    i1 = merge[1]
    i0 = merge[0]
    
    assert i1 > i0, 'Second index must be greater than first index'
    
    mol1 = molecules.pop(i1)
    mol0 = molecules.pop(i0)
    
    mol1 = list(mol1)
    mol0 = list(mol0)
    
    s1 = starts.pop(i1)
    s0 = starts.pop(i0)
    
    e1 = ends.pop(i1)
    e0 = ends.pop(i0)
    
    starts.append(min(s0,s1,e0,e1))
    ends.append(max(s0,s1,e0,e1))
    
    mol = copy.deepcopy(mol1)
    mol[s0:e0] = mol0[s0:e0]
#     print(''.join(mol0))
#     print(''.join(mol1))
#     print(''.join(mol))
    if '.' in mol:
        molecules.append(''.join(mol))
    else:
        print('Completed ', ''.join(mol))
        complete.append(''.join(mol))
    print(len(molecules))

# Now we have a matrix containing all the overlaps between pairs of molecules.

# We begin by greedily combining reads with the greatest overlap

000000000000000
00000000000000000
17 32
0 17
000000000000000
00000000000000000
{17}


____
000000000000000
00000000000
17 32
0 11
000000000000000
00000000000
17 32
0 11
000000000000000
000000000000
17 32
22 34
000000000000000
000000000000
{32, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
0000000000
0000000000
____
000000000000000
00000000000000000
17 32
5 22
000000000000000
00000000000000000
{17, 18, 19, 20, 21, 22}
00000
00000
____
000000000000000
000000000000000
17 32
16 31
000000000000000
000000000000000
{17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
00000000000000
00000000000000
____
000000000000000
0000000000000000
17 32
0 16
000000000000000
000000000000000
17 32
19 34
000000000000000
000000000000000
{32, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
0000000000000
0000000000000
____
000000000000000
000000000000000
17 32
4 19
000000000000000
000000000000000
{17, 18, 19}
00
00
____
000000000000000
000000000000
17 32
22 34
000000000000000
000000000000
{32, 22, 23, 24, 

KeyboardInterrupt: 

In [91]:
merged_molecules = molecules

In [109]:
['00000000000000000.................',
 '......................000000000000',
 '000000000000......................',
 '..0000000000000000000000..........',
 '....000000000000000...............',
 '....................00000000000000',
 '.................00000000000000000',
 '.......................00000000000',
 '00000000000000000000..............',
 '000000000000000000................',
 '0000000000000000..................',
 '000000000000000000................',
 '00000000000000000.................',
 '...............0000000000000000000',
 '000000000000......................',
 '.................00C00000000000000',
 '...............0A00000000000000000',
 '..............C0000000000000000000',
 '......................000000000C00',
 '................000000000000000C00',
 '...........00000A00000000000000C00',
 '.............000A00000000000000C00',
 '.......0000000C0000000000000000C00',
 '...................C00000000000000',
 '..................0C00000000000000',
 '.................00C00000000000000',
 '0000C00000000000000C0000000000000.',
 '.........0000000000C00000000000000',
 '0000000000000000..................',
 '0000000000000000A0000000..........',
 '......................000000000000',
 '...............0A00000000000000000',
 '000000000000000000................',
 '0000000000000000..................',
 '000000000000000...................',
 '000000000000000000000.............',
 '0000000000000000A00C..............',
 '00000000000000C00000000...........',
 '............0000000000000000000000',
 '....0000000000C.......000000000000',
 '00000000000000C000................',
 '......00000000C0000000000000000000',
 '......00000000C0000000000000000000',
 '000000000000000000................',
 '000000000000000000000.............',
 '00000000000000C000000000..........',
 '00000000000000000.................',
 '................000000000000000000',
 '00000000000000C00.................',
 '0000C0000000000000................',
 '00000000000000C000000000000.......',
 '00000000000000C0000000............',
 '0000C00000000.....................',
 '.................00000000000000000',
 '........000000000000000...........',
 '.................000..............',
 '000000000000000...................',
 '0000C00000000000A0................',
 '000000000000000000.000000000000000',
 '..............00000000000000000000',
 '0000C00000000000000C0000..........',
 '.........0000000000000000000000...',
 '........0000000000000000..........',
 '................000000000000000000',
 '0000000000000.....00000000000.....',
 '................000000000000000000',
 '..............0.....00000000000000',
 '....................00000000000000',
 '0000C0000000000000000.............',
 '.....................0000000000000',
 '0000C00000000000A0................',
 '000000000000......................',
 '0000000000000.....................',
 '........00000000000..0000000000000',
 '0000000000000000..................',
 '........0000000000.C00000000000C00',
 '000000000000..00000000000000000000',
 '00000000000000000000000000........',
 '......................000000000000',
 '0000000000000000..................',
 '00000000000000.........00000000000',
 '0000000000000000..................',
 '.....................0000000000000',
 '.....................0000000000000',
 '................000C00000000000000',
 '.......000000000000000000000......',
 '00000000000.......................',
 '.....00000000000000000000000000000',
 '000000000000000....000000000000000',
 '..............00000000000000000000',
 '.......................00000000000',
 '.........000000000000000..........',
 '...................00000000000....',
 '0000000000000.....................',
 '...........00000000000000000000000',
 '..................................',
 '.....00000000000A00000000000000C00',
 '.......0000000C.......000000000C00',
 '.............000.....0000000000000',
 '0000000000000000A00000000000......',
 '0000000000000000..................',
 '.....................0000000000000',
 '......................000000000000',
 '00000000000000000000..............',
 '0000000000000000..................',
 '00000000000000000.................',
 '.................00000000000000000',
 '0000000000000000000...............',
 '.............00000000.000000000000',
 '000000000000......................',
 '..................................',
 '...........00000000000000000000000',
 '0000000000000000000C.00000000000..',
 '...................000000000000000',
 '000000000000000...................',
 '0000000000000000000...0...........',
 '..................0000000000000C00',
 '00000000000000000000..............',
 '000000000000000000000.............',
 '.................0000000000000....',
 '..............00000000000000000...',
 '.........0000000000...............',
 '...............00000..............',
 '................000000000000000000',
 '.......................00000000000',
 '..................0000000000000C..',
 '0000000000000000000...............',
 '00000000000000....................',
 '0000...0000000000000000000000000..',
 '000000000000.....0................',
 '0000000000000000000000............',
 '....00000000000000000.............',
 '...........0000000000.............',
 '0000000000000000..................',
 '000000000000000000................',
 '00000000000000000000000...........',
 '00000000000000C00.................',
 '..................0000000000000C00',
 '........0000000000000.............',
 '.........000000000000000..........',
 '..................................',
 '0000000000000000000000000000......',
 '00000000000000000000..............',
 '.................00C00000000000000',
 '.................0000000000.......',
 '........000000000000..............',
 '.000C00000000..0000...............',
 '..............00A0000.............',
 '000000000000000..00000000000000000',
 '..............00000000............',
 '...00000000000000000000...........',
 '..............C00000000000000.....',
 '0000C000000000000..000000.........',
 '...0000000000000000000000000000C00',
 '0000000000000000A000..............',
 '................000000000000000C0.',
 '.....................0000000000C00',
 '.................00000000000000C00',
 '................A00000000000000000',
 '0000C000000.........00000000000000',
 '.000000000000000A00000000000000000',
 '.........0000000A00C00000000000000',
 '............00C0000000000000000000',
 '000000000000.....00000000000000000',
 '0000C00000000000000...............',
 '..00000000000000000000000000000000',
 '.......................00000000000',
 '........00000000000000000000000000',
 '00000000000000000.00..............',
 '0000C000000000000000..............',
 '..............00000000000000000000',
 '00000000000000000000000000000000..',
 '000000000000000...................',
 '0000000000000000..................',
 '.........00000000000000...........',
 '00000000000000000000000000........',
 '000000000000000000000000000000....',
 '............00C......0000000000000',
 '0000C0000000000...................',
 '..............00A00000000000000000',
 '..........00000000000000..........',
 '0000C0000000000000000.............',
 '.....00000000000000000000000000000',
 '0000000000000000000000............',
 '0000C00000000000..................',
 '0000000000000000000...............',
 '...0000000000000000000000.........',
 '00000000000000000000..............',
 '............0000000C00000000000000',
 '....C000000000000000000000000000..',
 '0000000000000000000...............',
 '0000000000000000000...............',
 '00000000000000000.................',
 '....................00000000000000',
 '0000C000000000000000000000000000..',
 '......0000000000000000000000000000',
 '0000000000000000A0................',
 '....000000000000000000000000000C00',
 '..............C0000000000000000000',
 '.......000000000000000000000000000',
 '0000000000000.....................',
 '...........00000000C000000000000..',
 '...................000000000000...',
 '.....00000000000000000000000000000',
 '..........000000A00000000000000C00',
 '00000000000000C000................',
 '...............000000000000000....',
 '..............00000C00000000000000',
 '000000000000000000................',
 '.................00000000000000C00',
 '...........00000000000000000000000',
 '...............0000000000000000000',
 '...................00000000000000.',
 '0000000000000000000000............',
 '........0000000000000000000000000.',
 '.......000000000000000000000000000',
 '...............0000000000000000C0.',
 '000000000000.........0000000000C00',
 '0000C0000000000000000000000000000.',
 '.........0000000000000000000000000',
 '0000C0000000000000................',
 '0000000000000......000000000000000']

['00000000000000000.................',
 '......................000000000000',
 '000000000000......................',
 '..0000000000000000000000..........',
 '....000000000000000...............',
 '....................00000000000000',
 '.................00000000000000000',
 '.......................00000000000',
 '00000000000000000000..............',
 '000000000000000000................',
 '0000000000000000..................',
 '000000000000000000................',
 '00000000000000000.................',
 '...............0000000000000000000',
 '000000000000......................',
 '.................00C00000000000000',
 '...............0A00000000000000000',
 '..............C0000000000000000000',
 '......................000000000C00',
 '................000000000000000C00',
 '...........00000A00000000000000C00',
 '.............000A00000000000000C00',
 '.......0000000C0000000000000000C00',
 '...................C00000000000000',
 '..................0C00000000000000',
 '.................00C000

In [111]:
complete

['0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '0000000000000000000000000000000000',
 '00000000000000C0000000000000000C00',
 '00000000000000C0000000000000000C00',
 '0000000000000000000000000000000000',
 '0000000000000000A00000000000000C00',
 '0000000000000000000000000000000000',
 '0000C00000000000000C00000000000000',
 '00000000000000000000000

In [64]:
x=[0,1,2]
y=[1,1,0]

starts=[1,0]
ends=[2,1]

i0=0
i1=1
mol = ['.']*len(x)
mol[starts[i0]:ends[i0]+1] = x[starts[i0]:ends[i0]+1]
mol[starts[i1]:ends[i1]+1] = y[starts[i1]:ends[i1]+1]

print(y[starts[i1]:ends[i1]])

[1]


In [113]:
print('Unassembled ',len(molecules))
print('Assembled ',len(complete))

Unassembled  222
Assembled  36


In [114]:
from collections import Counter

c = Counter(complete)

In [115]:
c

Counter({'0000000000000000000000000000000000': 28,
         '0000000000000000A00000000000000C00': 1,
         '00000000000000C0000000000000000C00': 2,
         '0000C00000000000000000000000000000': 2,
         '0000C00000000000000C00000000000000': 1,
         '0000C00000000000A00000000000000000': 2})

In [120]:
df = pd.read_csv('TestFile_1_basevector.txt',sep='\t')
m = set(df['Bases_vector'])

In [124]:
m.intersection(c)

{'0000000000000000000000000000000000',
 '0000000000000000A00000000000000C00',
 '00000000000000C0000000000000000C00',
 '0000C00000000000000000000000000000',
 '0000C00000000000000C00000000000000',
 '0000C00000000000A00000000000000000'}